# Import Library

In [ ]:
import tensorflow as tf
import numpy as np
import os
import shutil
import glob
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score
from sklearn.utils import shuffle
import random
import pandas as pd
import matplotlib.pyplot as plt
import cv2
keras.optimizers.Adam

keras.src.optimizers.adam.Adam

# Mengakses Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
base_dir = '/content/drive/My Drive/Tugas Akhir/dataset/dataset 1'
!ls "/content/drive/My Drive/Tugas Akhir/dataset/dataset 1"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
ls: cannot access '/content/drive/My Drive/Tugas Akhir/dataset/dataset 2': No such file or directory


# Menampilkan jumlah dataset

In [ ]:
cataract_dir = os.path.join(base_dir, 'cataract/')
normal_dir = os.path.join(base_dir, 'normal/')

print("Jumlah Citra Asli")
print('Jumlah dataset Cataract : ',len(os.listdir(cataract_dir)))
print('Jumlah dataset Normal : ',len(os.listdir(normal_dir)))

cataract_augmented = os.path.join(base_dir, 'cataract_augmented/')
normal_augmented = os.path.join(base_dir, 'normal_augmented/')

print(" ")
print("Jumlah Citra Hasil Augmentasi")
print('Jumlah dataset Cataract : ',len(os.listdir(cataract_augmented)))
print('Jumlah dataset Normal : ',len(os.listdir(normal_augmented)))

# num_cataract = len(os.listdir(cataract_dir))
# num_normal = len(os.listdir(normal_dir))

# # Menampilkan diagram jumlah
# labels = ['Cataract', 'Normal']
# values = [num_cataract, num_normal]

# plt.bar(labels, values)
# plt.xlabel('Kelas')
# plt.ylabel('Jumlah')
# plt.title('Jumlah Data Setiap Kelas')
# plt.show()

Jumlah Citra Asli


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Tugas Akhir/dataset/dataset 2/cataract/'

# Menggabungkan dataset

In [ ]:
dataset_dir = os.path.join(base_dir, 'dataset')
train_dir = os.path.join(base_dir, 'latih')
test_dir = os.path.join(base_dir, 'test')
validation_dir = os.path.join(base_dir, 'validasi')

# Split Dataset

In [ ]:
image_paths = []
labels = []
for label, label_dir in enumerate([cataract_dir, normal_dir, cataract_augmented, normal_augmented]):
    for filename in os.listdir(label_dir):
        image_paths.append(os.path.join(label_dir, filename))
        labels.append(label)

train_val_paths, test_paths, train_val_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42, stratify=labels)

# num_train_val = len(train_val_paths)
# num_test = len(test_paths)

# # Menampilkan diagram jumlah split
# labels = ['Train/Validation', 'Test']
# values = [num_train_val, num_test]
# plt.bar(labels, values)
# plt.xlabel('Split')
# plt.ylabel('Jumlah')
# plt.title('Jumlah Hasil Split')
# plt.show()

def my_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    f1_Score = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1_Score))
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1_Score

# Pembuatan Model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='softmax'),
])

# Compile

In [ ]:
learning_rate = 0.001
model.compile(loss='categorical_crossentropy',
        optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
        metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 74, 74, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 36, 36, 64)       

# Preprocessing dan Augmentasi

In [ ]:
dataset_path = "/content/drive/My Drive/Tugas Akhir/dataset/dataset 1/normal"
augmented_paths = "/content/drive/My Drive/Tugas Akhir/dataset/dataset 1/normal_augmented"

if not os.path.exists(augmented_paths):
    os.mkdir(augmented_paths)

#Konfigurasi augmentasi
train_datagen = ImageDataGenerator(
    # rescale=1./225,
    # brightness_range=[0.5, 1.0],
    # horizontal_flip=True,
    # vertical_flip=True,
    # shear_range=0.2,
    # rotation_range=45,
    # fill_mode='mirror',
)

# image_files = []
# for root, dirs, files in os.walk(dataset_path):
#     for file in files:
#         if file.endswith((".jpg", ".png")):
#             image_files.append(os.path.join(root, file))

# # Augmentasi dan menyimpan setiap citra
# for image_file in image_files:
#     img = cv2.imread(image_file)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Ubah warna citra jika diperlukan

#     # Koreksi gamma dengan nilai gamma yang diinginkan (misal: 1.5)
#     gamma_value = 1.5
#     img = adjust_gamma(img, gamma=gamma_value)

#     # Resize citra jika diperlukan
#     img = cv2.resize(img, (150, 150))

#     # Mengubah dimensi citra menjadi bentuk (1, height, width, channels)
#     img = np.expand_dims(img, axis=0)

#     # Dapatkan nama file asli
#     original_filename = os.path.basename(image_file)

#     # Membuat iterator augmentasi
#     aug_iter = train_datagen.flow(img, save_to_dir=augmented_paths, save_prefix=original_filename[:-4])

#     # Mendapatkan ekstensi citra asli
#     extension = os.path.splitext(original_filename)

#     # Melakukan augmentasi dan menyimpan citra
#     for i in range(1):
#         augmented_img = next(aug_iter)
#         # Menyimpan citra dengan ekstensi yang sama seperti citra asli
#         augmented_filename = f"{original_filename[:-4]}augmented{i}{extension[1]}"
#         cv2.imwrite(os.path.join(augmented_paths, augmented_filename), cv2.cvtColor(augmented_img[0], cv2.COLOR_RGB2BGR))

# Training Model

In [ ]:
class_labels = [cataract_dir, normal_dir]
skf = StratifiedKFold(n_splits=5, shuffle=True)

fold_num = 0
test_losses = []
test_accuracies = []
confusion_matrices = []

for train_index, val_index in skf.split(image_paths, labels):
    fold_num += 1
    print("Hasil Fold : ", fold_num)
    X_train, X_val = np.array(image_paths)[train_index], np.array(image_paths)[val_index]
    Y_train, Y_val = np.array(labels)[train_index], np.array(labels)[val_index]

    Y_train = Y_train.astype(str)
    Y_val = Y_val.astype(str)

    train_val_paths, test_paths, train_val_labels, test_labels = train_test_split(
        X_train, Y_train, test_size=0.2, random_state=42, stratify=Y_train
    )

    for each_index in range(len(X_val)):
        class_label = ''
        for i in range(len(class_labels)):
            if Y_val[each_index] == str(i):
                class_label = class_labels[i]
        shutil.move(
            os.path.join(base_dir, 'latih', class_label, X_val[each_index]),
            os.path.join(base_dir, 'validasi', class_label, X_val[each_index])
        )

    train_generator = train_datagen.flow_from_dataframe(
        pd.DataFrame({"image_path": train_val_paths, "label": train_val_labels}),
        x_col="image_path",
        y_col="label",
        target_size=(150, 150),
        batch_size=32,
        class_mode="categorical",
        shuffle=True
    )

    val_datagen = ImageDataGenerator(
        # rescale=1./225
        )

    val_generator = val_datagen.flow_from_dataframe(
        pd.DataFrame({"image_path": X_val, "label": Y_val}),
        x_col="image_path",
        y_col="label",
        target_size=(150, 150),
        batch_size=32,
        class_mode="categorical",
        shuffle=False
    )

    # Training Model
    history = model.fit(
        train_generator,
        epochs=25,
        steps_per_epoch=len(train_generator),
        validation_data=val_generator,
        validation_steps=len(val_generator),
        verbose=2
    )

    predictions = model.predict(val_generator, verbose=1)
    y_predictions = np.argmax(predictions, axis=1)
    true_classes = val_generator.classes

    # evaluate validation performance
    print("*** Performance on Validation data ***")
    val_acc, val_prec, val_fScore = my_metrics(true_classes, y_predictions)

Hasil Fold :  1
Found 4054 validated image filenames belonging to 4 classes.
Found 1268 validated image filenames belonging to 4 classes.
Epoch 1/25
127/127 - 24s - loss: 16.6112 - accuracy: 0.4378 - val_loss: 0.9932 - val_accuracy: 0.5221 - 24s/epoch - 192ms/step
Epoch 2/25
127/127 - 20s - loss: 0.9145 - accuracy: 0.5557 - val_loss: 0.8281 - val_accuracy: 0.5552 - 20s/epoch - 161ms/step
Epoch 3/25
127/127 - 22s - loss: 0.7488 - accuracy: 0.6085 - val_loss: 0.6759 - val_accuracy: 0.6073 - 22s/epoch - 172ms/step
Epoch 4/25
127/127 - 22s - loss: 0.6884 - accuracy: 0.6455 - val_loss: 0.6584 - val_accuracy: 0.6341 - 22s/epoch - 172ms/step
Epoch 5/25
127/127 - 21s - loss: 0.6558 - accuracy: 0.6660 - val_loss: 0.6356 - val_accuracy: 0.6593 - 21s/epoch - 165ms/step
Epoch 6/25
127/127 - 21s - loss: 0.6066 - accuracy: 0.6961 - val_loss: 0.5608 - val_accuracy: 0.7295 - 21s/epoch - 164ms/step
Epoch 7/25
127/127 - 20s - loss: 0.5750 - accuracy: 0.7089 - val_loss: 0.5576 - val_accuracy: 0.7208 - 20

#Testing Model

In [ ]:
class_labels = [cataract_dir, normal_dir]
skf = StratifiedKFold(n_splits=5, shuffle=True)

fold_num = 0
test_losses = []
test_accuracies = []
confusion_matrices = []

for train_index, test_index in skf.split(train_val_paths, train_val_labels):
    fold_num += 1
    print("Hasil Fold : ", fold_num)
    X_train, X_test = np.array(train_val_paths)[train_index], np.array(train_val_paths)[test_index]
    Y_train, Y_test = np.array(train_val_labels)[train_index], np.array(train_val_labels)[test_index]

    for each_index in range(len(X_test)):
        class_label = ''
        for i in range(len(class_labels)):
            if Y_test[each_index] == str(i):
                class_label = class_labels[i]
        shutil.move(
            os.path.join(base_dir, 'latih', class_label, X_test[each_index]),
            os.path.join(base_dir, 'test', class_label, X_test[each_index])
        )

    test_datagen = ImageDataGenerator(
        # rescale=1./225
        )
    test_labels = [str(label) for label in Y_test]

    test_generator = test_datagen.flow_from_dataframe(
        pd.DataFrame({'image_path': X_test, 'label': test_labels}),
        x_col='image_path',
        y_col='label',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )

    test_loss, test_acc = model.evaluate(
        test_generator,
        steps=len(test_generator),
        verbose=1
    )

    test_losses.append(test_loss)
    test_accuracies.append(test_acc)

    predictions = model.predict(test_generator, verbose=1)
    y_predictions = np.argmax(predictions, axis=1)
    true_classes = test_generator.classes

    # evaluate performance on testing data
    print("*** Performance on Testing data ***")
    test_acc, test_prec, test_fScore = my_metrics(true_classes, y_predictions)
    confusion_matrices.append(confusion_matrix(true_classes, y_predictions))

Hasil Fold :  1
Found 811 validated image filenames belonging to 4 classes.
26/26 [==============================] - 4s 140ms/step
*** Performance on Testing data ***
Accuracy  : 0.9173859432799013
Precision : 0.9299296886899454
f1Score : 0.9167237147599102
[[130   3   0   0]
 [  7 130   0   0]
 [  0   0 266   0]
 [  0   0  57 218]]
Hasil Fold :  2
Found 811 validated image filenames belonging to 4 classes.
26/26 [==============================] - 3s 129ms/step
*** Performance on Testing data ***
Accuracy  : 0.9223181257706535
Precision : 0.9327735095568715
f1Score : 0.9219078733990393
[[132   1   0   0]
 [ 14 123   0   0]
 [  0   0 266   0]
 [  0   0  48 227]]
Hasil Fold :  3
Found 811 validated image filenames belonging to 4 classes.
26/26 [==============================] - 3s 104ms/step
*** Performance on Testing data ***
Accuracy  : 0.9223181257706535
Precision : 0.9322122425115607
f1Score : 0.9219367272339317
[[131   2   0   0]
 [ 14 123   0   0]
 [  0   0 266   0]
 [  0   0  47 2